# Geoprocessing Toolkit 1

In [ ]:
import os
import pandas as pd
import geopandas as gpd
%matplotlib notebook
from shapely.geometry import Point, LineString, Polygon
from fiona.crs import from_epsg
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Everything above here should basically be standard at this point!!!

![duh](https://media.giphy.com/media/aVtdz7iNVPI1W/giphy.gif)
________________________________________

Aite lets change the directory!

In [ ]:
os.chdir('data/colombia/')

Time to read the shapefile!! __geopandas is our library and then .read_file() is our function__

![duh](https://media.giphy.com/media/y4E6VumnBbIfm/giphy.gif)


In [ ]:
colo_shape = gpd.read_file('gadm36_COL_1.shp')

Wanna check it out? Use .plot() or .head()

In [ ]:
colo_shape.plot()

How about some tabular data? Go to 
* http://geo.aiddata.org/query/#!/
* Select Colombia GADM
* Adm 1 boundary
* Add to Request
    1. VIIRS Nighttime Lights - 2013 SUM
    2. Drug Cultivation Sites
    3. UCDP - 2014 SUM

And lets rename that long csv file. How about colo.csv. Don't forget to put it into the colombia folder!

In [ ]:
colo_tab = pd.read_csv('colo.csv')

Who needed encoding? Remeber what it was? 

Hint: y tu brutus?

Wanna check out this data? How about .describe(), dtypes, head/tail() 

In [ ]:
colo_tab.describe()

These variable names are terrible! Lets change them real quick.

First we will get a list of column names.

In [ ]:
colo_tab.columns

So its not every one that we need to change, just the ones that are long and we plan to use for some analysis.

In [ ]:
colo_tab.rename(columns = {'v4composites_calibrated_201709.2013.sum': 'lights',
       'drugdata_categorical_201708.none.categorical_count': 'drug_count',
       'drugdata_categorical_201708.none.categorical_none': 'drug_other',
       'drugdata_categorical_201708.none.categorical_cannabis': 'herb',
       'drugdata_categorical_201708.none.categorical_coca_bush': 'coke',
       'drugdata_categorical_201708.none.categorical_opium': 'heroin',
       'drugdata_categorical_201708.none.categorical_mix': 'drug_mix',
       'ucdp_deaths_171.2014.sum': 'ucdp_deaths'}, inplace = True)

Pretty easy right?!?!?

![chill](https://media.giphy.com/media/TlK63Euc9KArc2a0kEw/giphy.gif)

Lets merge the two!

What column matches up between the shapefile and the tabular file?

In [ ]:
colo_tab.head()

In [ ]:
colo_shape.head()

In [ ]:
colombia = colo_shape.merge(colo_tab, on='HASC_1', how='left')

In [ ]:
colombia.head()

Nice now we can get into actually looking at the data and using some of our geoprocessing tools!

# Selection

Remember we can use selection to get information about certain spatial objects or the distribution of data within these groups.

Lets start by selecting which of the first administrative districts in Colombia, also known as Departments, contains the capital. 

1. Go to https://www.google.com/maps
2. Type in the capital of Colombia (World Geography quiz!)
3. Right click on the city name (or center)
4. Click What's here?
5. Input the latitude and longitude as a point!

Lets make it as a pandas dataframe and use the same techique to make it a geodataframe too.

In [ ]:
data = {'city':['Bogota'], 'latitude':[4.5709], 'longitude':[-74.2973]}

In [ ]:
cities = pd.DataFrame(data)
city_geometry = [Point(xy) for xy in zip(cities.longitude, cities.latitude)]
city_gdf = gpd.GeoDataFrame(cities, geometry = city_geometry)

In [ ]:
colombia.crs = from_epsg(4326)
city_gdf.crs = from_epsg(4326)

In [ ]:
fig, ax = plt.subplots()
colombia.plot(ax=ax, facecolor='gray');
city_gdf.plot(ax=ax,color='red');
ax.set_aspect('equal')

![alrighty](https://media.giphy.com/media/5hc2bkC60heU/giphy.gif)

So we can see that the point for Bogota is in fact, within Colombia. How about we select the department it is within.

In [ ]:
for index, row in colombia.iterrows():
    if city_gdf.within(row['geometry']).any():
        print(row['NAME_1_x'])

Interesante! So now we know the department which has the capital city and we didn't even have to use google.

We did the previous selection using a geospatial __location__ technique.

Since we know the name of the department, now we can use an __attribute__ selection.

In [ ]:
cundinamarca = colombia.loc[colombia.NAME_1_x == "Cundinamarca"]

In [ ]:
cundinamarca

Nice! We have now done selection by both location and attribute. Lets move on to buffers.

# Buffers

So if we notice, the district of Bogota has no deaths recorded in 2014 within the UCDP data.  But that does not neccisarily mean violence is not proximate to the capital.  How about we buffer the city by some kilometers and see what the other districts look like.

In [ ]:
city_copy = city_gdf.copy()
city_copy['geometry'] = city_copy.geometry.buffer(2)

In [ ]:
fig, ax = plt.subplots()
colombia.plot(ax=ax, facecolor='gray');
city_copy.plot(ax=ax,color='red');
ax.set_aspect('equal')

Now that we have the buffer, lets find all the departments which intersect with this point.

We use the "intersects" function.

In [ ]:
for index, row in colombia.iterrows():
    if city_copy.intersects(row['geometry']).any():
        print("'" + str(row['NAME_1_x']) + "'")

In [ ]:
depts= ['Antioquia','Boyacá', 'Caldas','Caquetá','Casanare',
        'Chocó','Cundinamarca','Huila','Meta','Quindío',
        'Risaralda','Santander','Tolima','Valle del Cauca'
       ]

Here we can select all these departments using the loc and isin functions.

In [ ]:
depts_bogota = colombia.loc[colombia.NAME_1_x.isin(depts)]

In [ ]:
depts_bogota

In [ ]:
depts_bogota[['drug_count', 'ucdp_deaths']].describe()

In [ ]:
colombia[['drug_count', 'ucdp_deaths']].describe()

Now we can make comparisons between these two groups of departments within Colombia regarding descriptive spatial statistics.

# Raster

On to the raster data.  This data is slightly different than the discrete vector data we have worked with thus far.  Today we will work on importing a single raster image, displaying it, and then clipping it using a mask.

In [ ]:
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
%matplotlib inline
import pycrs
import json

Many of you(probably all) will have issues with the imports.

Use conda install package in the anaconda prompt,
or use the !pip install package in the box within jupyter.

Lets start with just loading the world map we have stored in our data folder. Since we changed to the subfolder of colombia, we will need to go backwards in the directory.  We can do this with the .. as you see below.

In [ ]:
world = gpd.read_file('../worldmap/cshapes.shp') 
colombia_full = world.loc[world.CNTRY_NAME == "Colombia"]

Ok now we use the rasterio (stands for raster input output) package and the function open, to get our raster image. These are .tif files.

In [ ]:
raster1 = rasterio.open('noaa/avh_1.tif')

Just like that we have our first raster!

You can find a huge repository of free and open-source rasters here:

https://earthexplorer.usgs.gov/


Lets just show this image as is.

In [ ]:
show(raster1)

Interesting, but can we change the color scheme?

Yes! Just like we did with the choropleth maps we can use the cmap choices.

In [ ]:
show(raster1, cmap='terrain')

![lit](https://media.giphy.com/media/jba8ucWVAhG9VcUkx9/giphy.gif)

Alright lets do something else, what if we don't want the entire raster but instead just a small portion.  Since the lab before we have focused on Colombia, lets go ahead and use that as our focus.

First step is we can define a function to get the geometry of the mask to then apply it.

In [ ]:
def getmask(polygon):
    
    return [json.loads(polygon.to_json())['features'][0]['geometry']]

Now lets use the full shapefile of Colombia.

In [ ]:
coords = getmask(colombia_full)

Now we can use the mask function we imported to actually get the images themseleves.

In [ ]:
out_img, out_transform = mask(raster1, shapes=coords, crop=True)

In [ ]:
out_meta = raster1.meta.copy()

Now lets name the clipped raster file something, so we can actually write it into our folder.

In [ ]:
out_tif = r"clipped_colombia.tif"

In [ ]:
with rasterio.open(out_tif, "w", **out_meta) as dest:
    dest.write(out_img)

In [ ]:
clipped = rasterio.open(out_tif)

In [ ]:
show(clipped)

Finally lets make our plot a little prettier!

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
show(clipped, cmap='Blues', ax=ax)
plt.title("Colombia NDVI Raster")

### We did it!!

![excited](http://giphygifs.s3.amazonaws.com/media/nNxT5qXR02FOM/giphy.gif)